In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
import matplotlib.pyplot as plt
from datetime import datetime
import io
from openpyxl.utils import get_column_letter

In [2]:
df = pd.read_csv(r'transactions.csv')

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter out "Credit Card Payment" category
df = df[df['Category'] != 'Credit Card Payment']

In [3]:
# Filter out credit card payments and separate debit transactions as expenses and credit transactions as income
expenses_df = df[df['Transaction Type'] == 'debit'].copy()
income_df = df[df['Transaction Type'] == 'credit'].copy()

In [4]:
# Get unique categories from both expenses and income
unique_categories_expenses = set(expenses_df['Category'])
unique_categories_income = set(income_df['Category'])

In [5]:
# Set up a function to create budget Excel files and a dashboard sheet
def create_budget_excel(expenses, income, time_period):
    # Pivot expenses transactions based on the time period (weekly, monthly, yearly)
    pivoted_expenses = pd.pivot_table(expenses, index='Category', columns=pd.Grouper(key='Date', freq=time_period), values='Amount', aggfunc='sum', fill_value=0)
    
    # Pivot income transactions based on the time period (weekly, monthly, yearly)
    pivoted_income = pd.pivot_table(income, index='Category', columns=pd.Grouper(key='Date', freq=time_period), values='Amount', aggfunc='sum', fill_value=0)
    
    # Create a new Excel workbook
    wb = Workbook()
    
    # Create Transactions sheet for expenses
    ws_transactions_expenses = wb.create_sheet(title="Transactions (Expenses)")
    ws_transactions_expenses.append(expenses.columns.tolist())
    for row in dataframe_to_rows(expenses, index=False, header=False):
        ws_transactions_expenses.append(row)
    
    # Create Transactions sheet for income
    ws_transactions_income = wb.create_sheet(title="Transactions (Income)")
    ws_transactions_income.append(income.columns.tolist())
    for row in dataframe_to_rows(income, index=False, header=False):
        ws_transactions_income.append(row)
    
    # Create Summary sheet for expenses
    ws_summary_expenses = wb.create_sheet(title="Summary (Expenses)")
    for row in dataframe_to_rows(pivoted_expenses, index=True, header=True):
        ws_summary_expenses.append(row)
    
    # Create Summary sheet for income
    ws_summary_income = wb.create_sheet(title="Summary (Income)")
    for row in dataframe_to_rows(pivoted_income, index=True, header=True):
        ws_summary_income.append(row)
    
    # Create a dashboard sheet for top ten categories with highest expenses
    ws_dashboard_expenses = wb.create_sheet(title="Dashboard (Top Expenses)")

    # Calculate total expenses for each category
    total_expenses = pivoted_expenses.sum(axis=1)  # Sum along rows (categories)

    # Sort by expenses in descending order and select the top ten categories
    top_expense_categories = total_expenses.nlargest(10).index

    # Create a DataFrame for the top expense categories
    top_expense_data = pd.DataFrame({
        'Category': top_expense_categories,
        'Total Expenses': total_expenses[top_expense_categories]
    })

    # Add the top expense data to the worksheet
    for row in dataframe_to_rows(top_expense_data, index=False, header=True):
        ws_dashboard_expenses.append(row)

    # Add a table with total expenses, total income, and total savings
    total_expenses_value = total_expenses.sum()

    # Filter out categories with no income transactions
    total_income = pivoted_income.sum()
    total_income = total_income[total_income > 0]

    total_income_value = total_income.sum()
    total_savings_value = total_income_value - total_expenses_value

    ws_dashboard_expenses.append([])  # Add an empty row
    ws_dashboard_expenses.append(["Total Expenses", total_expenses_value])
    ws_dashboard_expenses.append(["Total Income", total_income_value])
    ws_dashboard_expenses.append(["Total Savings (Credit - Debit)", total_savings_value])

    # Save the workbook with a filename indicating the time period
    output_filename = f"budget_{time_period}.xlsx"
    wb.save(output_filename)
    print(f"Budget Excel file for {time_period} period saved as {output_filename}")

In [6]:
# Create budget Excel files and a dashboard sheet for different time periods
create_budget_excel(expenses_df, income_df, 'W')  # Weekly
create_budget_excel(expenses_df, income_df, 'M')  # Monthly
create_budget_excel(expenses_df, income_df, 'Y')  # Yearly

Budget Excel file for W period saved as budget_W.xlsx
Budget Excel file for M period saved as budget_M.xlsx
Budget Excel file for Y period saved as budget_Y.xlsx
